Вариант 2
Необходимо собрать информацию по продуктам питания с сайтов:
Роскачество официальный сайт. Исследование качества продуктов питания | Рейтинг товаров.
Список протестированных продуктов на сайте Росконтроль.рф
Получившийся список должен содержать:
Наименование продукта.
Категорию продукта (например «Бакалея»).
Подкатегорию продукта (например «Рис круглозерный»).
Параметр «Безопасность».
Параметр «Качество».
Общий балл.
Сайт, откуда получена информация. ### Структура должна быть одинаковая для продуктов с обоих сайтов. Общий результат можно вывести с помощью dataFrame через Pandas.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas

In [2]:
response = requests.get('https://rskrf.ru/ratings/produkty-pitaniya/').text

In [3]:
soup = BeautifulSoup(response, 'lxml')

In [4]:
categories = soup.find_all('div', {'class': 'category-item'})

In [5]:
categoiesDict = {}
subCategoriesDict = {}
productsDict = []
data = []

for category in categories:
    categoryName = category.find('span', {'class': 'h5'}).text
    link = category.find('a').get('href')
    categoiesDict[categoryName]=link
    
    response = requests.get('https://rskrf.ru/' + link).text
    soup = BeautifulSoup(response, 'lxml')
    
    subCategories = soup.find_all('div', {'class': 'category-item'})
    
    
    for subCategory in subCategories:
        subCategoryName = subCategory.find('span', {'class': 'd-xl-block d-none'}).text
        link = subCategory.find('a').get('href')
        subCategoriesDict[subCategoryName]=link
        
        response = requests.get('https://rskrf.ru/' + link).text
        soup = BeautifulSoup(response, 'lxml')

        productLinks = soup.find_all(href=re.compile("/goods/*"))

        for productLink in productLinks:
            product = productLink.get('href')
            #data.append([categoryName, subCategoryName, product])
            
            response = requests.get('https://rskrf.ru/' + productLink.get('href')).text
            soup = BeautifulSoup(response, 'lxml')
            
            try:
                productTitle = soup.find('p', {'product-subtitle'}).text

                ratings = soup.find_all('div', {'class': 'rating-item'})
                for rating in ratings:
                    if rating.find(string=re.compile("Общий рейтинг")) != None:
                        total = rating.find_all('span')[1].text
                for rating in ratings:
                    if rating.find(string=re.compile("Безопасность")) != None:
                        safety = rating.find_all('span')[1].text
                for rating in ratings:
                    if rating.find(string=re.compile("Качество")) != None:
                        quality = rating.find_all('span')[1].text
                data.append([productTitle.strip(), categoryName, subCategoryName, safety, quality, total])
            except:
                pass
    print(data)


[['Fine Life Крупа рисовая. рис шлифованный. первый сорт', 'Бакалея', 'Рис круглозерный', '5.000', '5.000', '5.0000'], ['Крупа Рисовая Makfa. Рис шлифованный круглозерный. первый сорт', 'Бакалея', 'Рис круглозерный', '5.000', '5.000', '5.0000'], ['Д. Крупа рисовая шлифованная. Сорт 1', 'Бакалея', 'Рис круглозерный', '5.000', '5.000', '5.0000'], ['Крупа рисовая шлифованная, фасованная под торговым наименованием «Рис круглозерный «Дон Густо». Первый сорт.', 'Бакалея', 'Рис круглозерный', '5.000', '5.000', '5.0000'], ['Мистраль. Крупа рисовая фасованная. Белый круглозерный рис «Кубань». Первый сорт', 'Бакалея', 'Рис круглозерный', '5.000', '5.000', '5.0000'], ['Националь. Крупа рисовая шлифованная (рис круглозерный). Кубанский. Первый сорт.', 'Бакалея', 'Рис круглозерный', '5.000', '5.000', '5.0000'], ['Националь. Крупа рисовая шлифованная (рис круглозерный). Краснодарский. Первый сорт. Высшее качество', 'Бакалея', 'Рис круглозерный', '5.000', '5.000', '5.0000'], ['Пассим. Крупа, рис шлиф